<a href="https://colab.research.google.com/github/wybin4/systems-theory/blob/main/%D0%90%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F%20%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
from google.colab import runtime

#константы

highlighted_indicator = 'X5' #выделенный показатель
number_of_classes = 3 #количество классов
class_names = ['A', 'B', 'C']

#загружаем файл с данными
from google.colab import files
uploaded = files.upload()

#получаем название файла
for key in uploaded.keys():
    filename = key

#считываем как csv
import pandas as pd
df=pd.read_csv(filename)

print(df)

#получаем названия столбцов = вершин графа
col_keys = list()
for i in df:
  col_keys.append(i)

#классы объектов
classes = list()
if highlighted_indicator in col_keys:
  h_i_values = df[highlighted_indicator].to_list()
  if number_of_classes > 0:
    h = (max(h_i_values) - min(h_i_values)) / number_of_classes
    for i in range(1, number_of_classes + 1):
      classes.append([j + 1 for j, val in enumerate(h_i_values) if val <= min(h_i_values) + i * h and val >= min(h_i_values) + (i - 1) * h])
  else:
    print('Укажите количество классов')
else:
  print('Выделенного показателя нет в списке')
  runtime.unassign()
for i in range(len(class_names)):
  print('Класс', class_names[i], '- ', end="")
  print(*classes[i], sep=', ')

#координаты центров тяжести

center_coords = list()
aver = 0
count = 0
for i in range(number_of_classes):
  center_coords.append([])
for i in col_keys:
  if i != highlighted_indicator:
    for k in classes:
      for l in k:
        aver += df[i][l - 1]
      aver /= number_of_classes
      aver = int(aver)
      center_coords[count % number_of_classes].append(aver)
      count += 1 
      aver = 0
print()
print('Координаты центров тяжести:')
for i in range(len(class_names)):
  print(class_names[i], '(', end="")
  print(*center_coords[i], sep=', ', end="")
  print(')')


#загружаем файл с апостериорными данными
uploaded = files.upload()

#получаем название файла
for key in uploaded.keys():
    filename = key

#считываем как csv
import pandas as pd
posterior_data=pd.read_csv(filename)

print(posterior_data)

#получаем названия столбцов = вершин графа
posterior_col_keys = list()
for i in posterior_data:
  posterior_col_keys.append(i)


#получаем названия строк
data_top = posterior_data.head()
rows = list()
for row in data_top.index:
    rows.append(row)
arr = list()

def find_min_indices(arr):
    min_val = min(arr)
    indices = [index for index, val in enumerate(arr) if val == min_val]
    return indices

def find_max_indices(arr):
    max_val = max(arr)
    indices = [index for index, val in enumerate(arr) if val == max_val]
    return indices

def print_classes(class_names, indices):
  count = 0
  if class_names:
    for i in indices:
      if len(i) > 1:
        for j in i:
            if (j != len(i) - 1):
              print(rows[count] + 1, '∈', class_names[i[j]], end=" или ")
            else:
              print(rows[count] + 1, '∈', class_names[i[j]])
      else:
        print(rows[count] + 1, '∈', class_names[i[0]])
      count += 1
  else:
    print(indices)

def print_rounded(letter, arr, m_numb, class_names):
  for i in range(len(arr)):
    print(letter + '(M'  + str(m_numb + 1) + ", " + class_names[i] + ")", end=" = ")
    if i != len(arr) - 1:
      print(round(arr[i], 2), end=", ")
    else:
      print(round(arr[i], 2)) 


print("Классификация от объектов до центров тяжести классов:")
arr = list()
mas = list()
indices = list()
for j in range(len(posterior_data)):
  for i in posterior_col_keys:
    arr.append(posterior_data[i][j])
  for k in range(len(center_coords)):
    mas.append(sum(list(map(lambda x, y: pow(x - y, 2), arr, center_coords[k]))))
  print_rounded('ρ', mas, j, class_names)
  indices.append(find_min_indices(mas))

  mas = list()
  arr = list()

print('Результат')
print_classes(class_names, indices)

print('Есть весовые коэффициенты?')
question = input()
if (question == 'да' or question == 'Да'):
  #загружаем файл с весовыми коэффициентами
  uploaded = files.upload()

  #получаем название файла
  for key in uploaded.keys():
      filename = key

  #считываем как csv
  import pandas as pd
  weight_coefficients=pd.read_csv(filename)

  print(weight_coefficients)

  #получаем названия столбцов = вершин графа
  weight_coefficients_keys = list()
  for i in weight_coefficients:
    weight_coefficients_keys.append(i)


  weights = list()
  for i in weight_coefficients:
    weights.append(weight_coefficients[i].to_list()[0])
  print()
  print("Классификация от объектов до центров тяжести классов с учетом весовых коэффициентов:")
  arr = list()
  mas = list()
  indices = list()
  a = list()
  for j in range(len(posterior_data)):
    for i in posterior_col_keys:
      arr.append(posterior_data[i][j])
    for k in range(len(center_coords)):
      a = list(map(lambda x, y: pow(x - y, 2), arr, center_coords[k]))
      for i in range(len(a)):
        a[i] = a[i] * weights[i]; #учитываем весовые коэффициенты
      mas.append(sum(a))
      a = list()
    print_rounded('ρ', mas, j, class_names)
    indices.append(find_min_indices(mas))
    mas = list()
    arr = list()

  print('Результат')
  print_classes(class_names, indices)


print()
print('Классификация по скалярному произведению:')
arr = list()
mas = list()
indices = list()
for j in range(len(posterior_data)):
  for i in posterior_col_keys:
    arr.append(posterior_data[i][j])
  for k in range(len(center_coords)):
    mas.append(sum(list(map(lambda x, y: x * y, arr, center_coords[k]))))
  print_rounded('β', mas, j, class_names)
  indices.append(find_max_indices(mas)) #максимальные элементы
  mas = list()
  arr = list()
print('Результат')
print_classes(class_names, indices)

print()
print('Классификация с использованием корреляционного метода:')
arr = list()
mas = list()
indices = list()
for j in range(len(posterior_data)):
  for i in posterior_col_keys:
    arr.append(posterior_data[i][j])
  for k in range(len(center_coords)):
    scalar_product = sum(list(map(lambda x, y: x * y, arr, center_coords[k]))) #вычисляем скалярное произведение
    mas.append(scalar_product - sum(center_coords[k]) * sum(arr) / len(center_coords[k])) #ф-ла корреляционного метода
  print_rounded('r', mas, j, class_names)
  indices.append(find_max_indices(mas)) #максимальные элементы
  mas = list()
  arr = list()
print('Результат')
print_classes(class_names, indices)

print()
print('Классификация по углу между векторами объекта и центра тяжести классов:')
arr = list()
mas = list()
indices = list()
for j in range(len(posterior_data)):
  for i in posterior_col_keys:
    arr.append(posterior_data[i][j])
  for k in range(len(center_coords)):
    scalar_product = sum(list(map(lambda x, y: x * y, arr, center_coords[k]))) #вычисляем скалярное произведение
    mas.append(scalar_product / (pow(sum(map(lambda x: x * x, center_coords[k])), 1/2) * pow(sum(map(lambda x: x * x, arr)), 1/2))) #вычисляем косинус

  print_rounded('cosφ', mas, j, class_names)
  indices.append(find_max_indices(mas)) #максимальные элементы
  mas = list()
  arr = list()
print('Результат')
print_classes(class_names, indices)

Saving априорные данные автоматическая класс.txt to априорные данные автоматическая класс (2).txt
   X1  X2  X3  X4  X5
0   9   6   7   2  10
1   6   7   2   3  40
2   3   2   3   7  80
3   8   5   9   2  20
4   4   4   6   9  80
5  10   4   8   2  10
6   7   9   3   6  50
7   5   8   1   6  40
8   5   3   6   8  90
Класс A - 1, 4, 6
Класс B - 2, 7, 8
Класс C - 3, 5, 9

Координаты центров тяжести:
A (9, 5, 8, 2)
B (6, 8, 2, 5)
C (4, 3, 5, 8)


Saving апостериорные данные автоматич клас — копия.txt to апостериорные данные автоматич клас — копия (2).txt
    X1   X2  X3   X4
0  7.5  6.5   5  3.5
1  8.0  8.0   1  5.0
2  8.0  8.0   1  5.0
Классификация от объектов до центров тяжести классов:
ρ(M1, A) = 15.75, ρ(M1, B) = 15.75, ρ(M1, C) = 44.75
ρ(M2, A) = 68.0, ρ(M2, B) = 5.0, ρ(M2, C) = 66.0
ρ(M3, A) = 68.0, ρ(M3, B) = 5.0, ρ(M3, C) = 66.0
Результат
1 ∈ A или 1 ∈ B
2 ∈ B
3 ∈ B
Есть весовые коэффициенты?
нет

Классификация по скалярному произведению:
β(M1, A) = 147.0, β(M1, B) = 124.5, β(M1, C) = 102.5
β(M2, A) = 130.0, β(M2, B) = 139.0, β(M2, C) = 101.0
β(M3, A) = 130.0, β(M3, B) = 139.0, β(M3, C) = 101.0
Результат
1 ∈ A
2 ∈ B
3 ∈ B

Классификация с использованием корреляционного метода:
r(M1, A) = 12.0, r(M1, B) = 6.38, r(M1, C) = -10.0
r(M2, A) = -2.0, r(M2, B) = 23.5, r(M2, C) = -9.0
r(M3, A) = -2.0, r(M3, B) = 23.5, r(M3, C) = -9.0
Результат
1 ∈ A
2 ∈ B
3 ∈ B

Классификация по углу между векторами объекта и центра тяжести кла